In [1]:
import scrapy
from scrapy.selector import Selector
from qna_crawler.items import QnaCrawlerItem

from collections import defaultdict
import sys
import datetime
import time
from fake_useragent import UserAgent
import random
import datetime
import re
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.chrome.options import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

import gspread
from oauth2client.service_account import ServiceAccountCredentials

from qna_crawler.spiders.lecture_scraper import *
import surfer

In [5]:
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

json_file_name = "woven-arcadia-269609-12b95dbdd1c3.json"

credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1YEO-EhcPmtj0r0YJzy-xNF6oVEdb8QHS43Eusck83so/edit#gid=2122150374"

# 스프레스시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)

# 시트 선택하기

ua = UserAgent()
co = webdriver.ChromeOptions()
co.add_argument("/Users/yoonhoonsang/Desktop/internet_lecture/qna_crawler/chromedriver")
co.add_argument("log-level=1")
#co.add_argument("headless")
co.add_argument("user-agent={}".format(ua.random))
co.add_argument("lang=ko_KR")
co.add_argument("--window-size=1920x1080")


worksheet = set_teachers('etoos_yhk_tcc', doc)

In [6]:
b = webdriver.Chrome(chrome_options=co)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [4]:
base_page = 'http://www.megastudy.net/teacher_v2/ChrReview/teacher_dom_review.asp?tec_cd=rimbaud666&LeftMenuCd=11&tec_type=1&HomeCd=&page={}'

In [5]:
page = 1

In [6]:
b.get(base_page.format(str(page)))

In [23]:
writer = b.find_element_by_css_selector(".review--viewName").text
date = b.find_element_by_css_selector(".review--viewDate").text
content = b.find_element_by_css_selector(".review--viewCon").text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".review--viewName"}
  (Session info: chrome=80.0.3987.132)


In [ ]:
tbody = b.find_elements_by_tag_name("tbody")[1]
rows = tbody.find_elements_by_tag_name("tr")
page = 30

while True:
    for index in range(len(rows)):
        try:
            print('start')
            print('dealing with page {} {}'.format(str(page), str(index+1)))
            b.get(base_page.format(str(page)))
            time.sleep(2)
            tbody = b.find_elements_by_tag_name("tbody")[1]
            rows = tbody.find_elements_by_tag_name("tr")
            row = rows[index]

            title_cover = row.find_elements_by_css_selector("td.subject")[0]
            title_cover_text = title_cover.text
            print('title: {}'.format(title_cover_text))
            if title_cover.text == '비공개 처리된 게시물입니다.':
                continue

            title = title_cover.find_element_by_tag_name('a')
            title.click()
            time.sleep(2)

            b.switch_to.frame(1)
            writer = b.find_element_by_css_selector(".review--viewName").text
            date = b.find_element_by_css_selector(".review--viewDate").text
            content = b.find_element_by_css_selector(".review--viewCon").text

            print(writer)
            print(date)
            print(content)

            worksheet.append_row([writer, date, title_cover_text, content])
            b.switch_to.default_content()


            b.execute_script("window.history.go(-1)")
            print('Go back to the main')
            time.sleep(3)
        except:
            pass
    page += 1
    
# p.85까지

start
dealing with page 30 1
title: 영어 수능 1등급 가자
이름 : 이유정   (ca39****) 자연계
등록일 : 2020-01-30  |  조회 : 50
선생님 강의 듣고 1등급 맞아볼께요!!0
Go back to the main
start
dealing with page 30 2
title: 너무 좋아요
이름 : 이용준   (dlrl****) 인문계
등록일 : 2020-01-30  |  조회 : 22
수능 영어를 어떻게 접근해야하고 어떻게 준비해야 되며 어떤 자세로 해야 할지 너무나 잘 알려주셔서 좋았습니다.
Go back to the main
start
dealing with page 30 3
title: 왜 이제 알았을까요 ㅜㅜ
이름 : 엄현정   (gree****) 인문계
등록일 : 2020-01-30  |  조회 : 55
진짜 이런강의를 만들어주셔서 너무 감사해요 ㅜㅜ 막 영어 포기하면 인생포기하는거다 그러는데 이 강의정도면 인생포기 안해도 될꺼 같아요 너무 개념부터 차근차근 진행되는게 쌤만의 강의 특징이지만 너무 완벽 그자체입니다
Go back to the main
start
dealing with page 30 4
title: 조정식이 조정식했다. 왜이말이 나오는지 이해가 될껍니다.
이름 : 오유빈   (dbql****) 자연계
등록일 : 2020-01-30  |  조회 : 65
사회인이였다가. 올해새로준비하는 재수생입니다
거의노베이스 기준이라 타s사 ㅈㅎㅊ선생님 강좌를 듣다가 도저히 제기준에는 맞지않아서.
선생님을 들었습니다...평소에 상위권위주 강좌선생님이라고 생각했었는데 그건 일반화였을뿐
강의력 필기 설명을 진짜 잘해주시고 제기준 모든과목 강의력 1타라고 할만큼
그리고 작년 시작해를 듣다가 앞에 부분이 너무...머랄까 부족한느낌이 없지않았는데
괜찮아 문장쉬운버전같이 올해는 리뉴얼 되서 더좋더라고요
대충 품사편 문장편 나누어져있는데 품사편은 수업을 듣기위해 반드시 있어야할도구들 원래알아야하는것
문장편은 괜찮아 e

start
dealing with page 32 4
title: 들을까 고민하시는 분들께,영어문장 잘 읽고싶다면 그냥 필.수.
이름 : 김지현   (skyh****) 인문계
등록일 : 2020-01-25  |  조회 : 269
작년에도 들어본 사람으로서 믿어봐는 진짜...그냥 최고에요ㅠㅠ필수다 필수.제가 조정식 선생님 강의를 듣기전에 혼자 영어 읽었을땐 '진짜 이런게 필요하면 공부하기 좋을텐데..','이건 뭔 소리야?'하는게 정말 많았어요ㅠㅠㅠㅠ그냥 영어 진짜 못읽었습니다 근데 지금은 술술 읽히는것도 많고 성적도 올랐고.. (물론 작년에도 해서 그렇겠지만)올해도 조정식쌤입니다 ㅎㅎ 요점은 열심히 하시면 누구나 가능하다는걸 말씀드리고 싶네요! 믿어봐책은 진짜 너무 좋아요 옆에 tip이라는것도 있고 올해는 복습북도 따로 있는데 그 책에 해석 끊어서 다 해주셨음 ㅠㅠㅠㅠㅠㅠㅠ그저 갓정식..그냥 영어하시는 분들에게 다 추천하고싶네요.호들갑이 아니라.. 아무튼 선생님 사랑해요 열심히하겠습니다!!!!!!!!!
Go back to the main
start
dealing with page 32 5
title: 조정식 선생님은
이름 : 강서영   (kase****) 인문계
등록일 : 2020-01-24  |  조회 : 119
사실 고3에게 응원과 많은 생각을 하게 하는 말들은 사치이고 힘이 안 된다고 생각하고 최대한 빡세게 공부하고 싶은 수험생에게 최고입니다. 냉철하면서도 가장 효율적인 강의를 제공해주시면서 나무같은 묵직함을 지녀서 힘들때 보면 뭔가 에너지가 들어오는 것 같은 분이십니다. 지칠 틈이 없어요. 감사합니다
Go back to the main
start
dealing with page 32 6
title: 이 사람은 혁신가다 (1)
이름 : 조성원   (chog****) 인문계
등록일 : 2020-01-24  |  조회 : 347
내가 생각해 본 조정식 강의에 대한 평가

일단 수능영어 78 받고 다시 공부 시작한 재수생임
아직은 확실해가 안 올라와서 괜찮아, 믿

Go back to the main
start
dealing with page 33 9
title: 어휘편에 이어 문장편 완강했습니다!
이름 : 이수빈   (casi****) 자연계
등록일 : 2020-01-22  |  조회 : 185
어휘편을 다 듣고 문장편을 듣다보니 중간중간 어휘편에서 말씀하신 부분이 다시 떠올라서 좋았습니다. 처음에는 예문을 잘 끊어가지 못하고 버벅이다가 어느정도 중간중간 멈추고 끊어보고 다시 설명을 들으면서 나중에 맞게 끊고 고개도 끄덕일 수 있었습니다. 복습할 것들이 계속 늘어났지만 그래도 복습을 계속 할수록 더 빨리 떠오르고 이해하고 있어요! 이제 믿어봐 편으로 넘어가서 인강 듣고 틈틈히 복습하겠습니다! 감사해요!!
Go back to the main
start
dealing with page 33 10
title: 선생님 감사합니다ㅜㅜ
이름 : 김수현   (ksh0****) 인문계
등록일 : 2020-01-22  |  조회 : 138
영어 1등급을 굳히기 위해서 많이 고민하고 했는데 이걸 듣고 고민이 없어져 가는 중이에요.
Go back to the main
start
dealing with page 34 1
title: 어휘편에 이어 문장편 완강했습ㄴ
이름 : 이수빈   (casi****) 자연계
등록일 : 2020-01-22  |  조회 : 133

Go back to the main
start
dealing with page 34 2
title: 아는 것같은 단어도 한번더! 꺼진 불도 다시보자
이름 : 왕기민   (mong****) 인문계
등록일 : 2020-01-22  |  조회 : 106
어렸을때부터 단어를 그저 일대일 대응으로 외웠기 때문에 조금 다른 의미가 나오면 문제를 틀리기 쉽더라구요. 또 새로운 의미를 외울려해도 원래 단어만 계속 생각났었는데 괜찮아 어휘편이 도움이 많이 됐습니다. 특히 전치사 쪽 of와 against는 원래 의미로는 추론하기 어려운 내용을 그림이나 범주화를 통해 이해하는 방법이 좋았습니다. 처음

Go back to the main
start
dealing with page 36 1
title: 다른 누구와도 비교할 수 없는 불후의 띵강입니다..
이름 : 이윤홍   (lyho****) 인문계
등록일 : 2020-01-17  |  조회 : 310
저는 영어 공부를 1,2학년 동안 제일 싫어했고 단어 암기, 구문 분석만 꾸준히 해서 겨우 2등급 중 후반대에 왔다갔다하게 만들었는데 메가 패스 구매 후 조정식 선생님 강의를 접하니 집에서 쳐도 모의고사 등급은 딱 2등급 그 정도였는데 첫 문장 잡아서 읽기 등 독해의 본질을 깨우치니 1등급을 드디어 연속으로 따냈습니다. 비록 제가 집에서 친 모의고사라서 실전과의 괴리감이 있을 지도 모르지만 결국 이것이 실전에서 나오도록 훈련을 하면 곧 그것이 실력으로 변하리라 믿습니다. 제가 고1 5등급에서 고2 2등급까지 올렸는데 아무리 구문이 탄탄하고 단어가 탄탄해도 글 읽는 속도가 느리고 영어 그 자체로 반응 못한다면 꽝인거였습니다. 전 항상 전문을 한국말로 번역해서 읽고 그래서 시간 지체가 심했었는데 글 읽는 법의 태도만 바꾸었더니 새로운 세계가 보입니다. 조정식 선생님을 잘 모르는 상위 대학 화석들과 더불어 상위 대학 졸업생들은 단 하나의 선생님을 꼽을건데 현재는 시대가 많이 변했습니다. 단언컨데 조정식 선생님 강의는 절대 평가 체제의 영어에서 최고의 효율성을 얻을 수 있을 것 같습니다.
Go back to the main
start
dealing with page 36 2
title: 조정식 쌤 정말 좋긴한데요...
이름 : 박성철   (qwmk****) 인문계
등록일 : 2020-01-16  |  조회 : 1,019
쌤 수업 정말 이해하기 쉽고 지금까지 배워온 영어 수업중에서 최고인데, 질문을 했는데 3일이 지났는데 답변이 안오네요... 쓸데없는 질문도 아니고 문법 헷갈려서 질문 해봤는데 아직까지 안오네요... 제질문이 쓸데없는건지 피드백 이라도 해주면 좋겠네요...질문만 빨리 답해주기면 별 5개도 모자란 수업입니다

Go back to the main
start
dealing with page 38 6
title: 정말 도움이 많이 되는 강좌
이름 : 엄선명   (mksu****) 자연계
등록일 : 2020-01-15  |  조회 : 38
괜찮아 시리즈를 모두 수강하고 있는데
이 어휘편은 정말로 도움이 많이 되었습니다
솔직히 영어공부하면서 가장 힘든게 단어인데 그 점을
선생님께서 정확하게 캐치하셔서 도움을 많이 받을 수 있던 강좌입니다
수능 영어를 공부하는 학생은 다 들어봤으면 좋겠어요
Go back to the main
start
dealing with page 38 7
title: 기초를 쌓는데 매우 도움이 많이 되는 강좌입니다
이름 : 엄선명   (mksu****) 자연계
등록일 : 2020-01-15  |  조회 : 52
괜찮아!문장편을 방금 완강하고 왔는데요
강의를 들으면서 기초가 아주 차곡차곡 잘 정리가 되는 느낌을 받을 수 있었습니다
그렇게 고난이도 해석이나 문제를 풀이하는 강좌는 아니어서 최상위권 학생들보다
수능 영어를 이제 막 시작한 학생들이 수강하면 효과를 많이 볼 수 있을 것 같아요
Go back to the main
start
dealing with page 38 8
title: 정말 좋은 것 같아요!
이름 : 이채연   (gidr****) 자연계
등록일 : 2020-01-15  |  조회 : 20
진짜 간신히 영어를 하는 예비 고3입니다! 조정식선생님께서 괜찮아 강좌를 수강을 한 이후로 영어 기출문제를 다 맞추지는 못하더라도 해석이 가능한 수준이 되었습니다!
Go back to the main
start
dealing with page 38 9
title: 이런 어휘강의는 처음 들어봤는데
이름 : 소현준   (hjso****) 자연계
등록일 : 2020-01-15  |  조회 : 590
평소에 단어당 뜻 하나씩 외우는게 완전히 습관화되어있던 저로서는 단어의 중심 의미 하나로 파생되어나가는 과정이 무척 흥미롭게 느껴졌습니다. 그리고 이렇게 예문 중심으

...

title: 6평 97-->9평 100 이 강의는 갓입니다
이름 : 최종욱   (jong****) 자연계
등록일 : 2019-09-05  |  조회 : 437
영어를 잘 모르고 어려워하던 학생이었는데 이 강의 4개 들은 이후에 성적이 수직 상승하게 됬습니다. 1회독 할수있을지는 모르겠는데 열심히 들어볼게요. 감사합니다
start
dealing with page 74 4
title: 작년 수능 80점 턱걸이에서 9모 1등급 받았어요
이름 : 손진희   (gnee****) 
등록일 : 2019-09-05  |  조회 : 424
도저히 가만히 있을수가 없어서 수강평 쓰러왔습니다
저는 반수생이라 공부를 늦게 시작했는데 하필 취약 과목이 영어였습니다 대학 친구들이 다 확실해를 추천해주더라고요 그래서 ebs 강좌랑 확실해 강좌를 병행하고 기출 공부를 같이 했습니다
확실해 순삽은 절대 흔들림 없는 문제해결 방식을 전수해주는 강좌인 것 같아요
"클리닉"강좌라는 말이 정말 맞아요 내가 그동안 풀던 방식이 말도안되는 개논리에다 운빨끼워맞추기 식이라는 걸 알게 되고 태도 교정 되고 루틴을 확립할 수 있었습니다 그리고 중간에 계속 정리해주시는 어휘 구문 문법이 정말 도움이 많이 되었습니다 문법문제도 스무스하게 풀었어요 쌤 덕에 순삽 다 맞았어요 감사합니다
(완강을 했는데 이게 초기화 비슷하게 되가지고 진도율이 저따구네요...)
start
dealing with page 74 5
title: 드디어 2등급ㅠㅠㅠ
이름 : 장선아   (paks****) 
등록일 : 2019-09-04  |  조회 : 182
안녕하세요 정식쌤ㅠㅠㅠ 9모 체점하고 놀라서 수강평쓰러 왔답니다ㅠㅠ진짜 뼛속까지 이과인 저로선 항상 수학이랑 과탐을한다고 영어에 대해 소홀하게 생각하고 있던 학생 중 한 명입니다ㅠㅠ항상 영어는 감으로 풀지~~이런 생각으로 지내서 만년 4등급인 학생이었답니다....수능특강 분석 때부터 수능완성 분석까지 항상 쌤께서 강조하셨던 순서대로 본문의 주제를 까먹지 않게 주기적으로 반복해서 봤습

start
dealing with page 76 3
title: 연계체감 처음느낀 후기
이름 : 송한결   (song****) 
등록일 : 2019-08-30  |  조회 : 264
재수생이라 영어를 놓기는 그래서 EBS공부를 시작하긴 했지만 솔직히 EBS 700지문중에 7지문 나오는거 현실적으로 큰 도움이 되겠냐싶기도 하고, 그냥 EBS책으로 영어공부 한다고 생각하고 꾸역꾸역 공부했습니다
예습은 필요없다고 하셨지만 어차피 계속 일시정지하게 되길래 강의듣기전에 혼자 해석해보고 단어찾고 까다로운 표현 줄치고 어려운 구문 표시하고.. 한강당 예습만 1시간 넘게 걸린거 같습니다
생각보다 영어에 시간이 너무 많이 들어가서 지치던 차에, 얼마전 사설모의고사를 봤습니다
6평 이후에 7월 들어서 EBS를 시작한 터라, EBS가 연계되는 첫번째 모의고사였습니다
결과는? 2년전 고2 6월인가 이후로 처음 받아보는 1등급이었습니다
29번부터 계속 눈에 익은 지문들이 쭈루룩 나와서 평소에는 29번부터 40번까지 시간과의 싸움이었는데 호롤ㄹ로 풀면서 영어시간에 여유를 처음 느껴본거같네요
7월에 수특 듣고, 8월에 영독 들었는데(복습도 다 못했는데) 이정도면 복습 빡세게하고 들어갈 수능이 기대됩니다 진짜
얼른 수완까지 마무리하고 좋은 결과 낼게요 빛정식 화이팅
start
dealing with page 76 4
title: 한줄 요약; 영어 구조가 보임
이름 : 유성민   (usm7****) 
등록일 : 2019-08-28  |  조회 : 733
영어 구조가 보이기 시작합니다
수능을 별로 안남은 이 시점에서 수능 공부를 시작하는
현역 수강생입니다
주변에 만류에도 불구하고 남은 시간동안 남들처럼 놀기보다 지금이라도 시작해서 보통이라도 따라잡고싶은 마음에 이렇게 시작했는데
조정식 선샹님 강위를 듣기 아주 잘했다고 생각이 되네요
감사합니다!

추가적으로 저는 영어 구조에대한 공부를 안하고 고등영어로 바로 건너뛰어 공부했더니 해석이 남들과 다른점이 많았습니다,, 근데 이 강의듣고 아 내가 

title: 레알 갓정식
이름 : 백민석   (keot****) 기타
등록일 : 2019-08-20  |  조회 : 133
빈칸 추론 선지 소거법도 알려주시고 정말 좋은거 같아요.. 정말 조정식쌤은 미쳐써요ㅠㅠ
start
dealing with page 79 1
title: 최고의 learningmate
이름 : 이재원   (ljw2****) 자연계
등록일 : 2019-08-19  |  조회 : 185
조정식 모의고사 해설강의를 들었습니다
저는 빈칸추론을 정말 못하는데 이번 모의고사에서도 하나를 찍었거든요
그런데 선생님께서 정확히 선지 소거하는 방법부터 차근차근 알려주시고
문제에 적용해주셔서 좋았어요
이 모의고사는 정말 푸는게 다가 아니라 선생님과 함께 가장
확실한 근거를 잡고 그 근거를 어떻게 잡아야 하는지 고민해보는 과정이 정말 중요하다고 생각합니다
많은 공부가 된 것 같네요
역시 조정식입니다.
start
dealing with page 79 2
title: 진짜 갓정석
이름 : 최건   (choi****) 인문계
등록일 : 2019-08-19  |  조회 : 126
여러방면으로 생각하게 해준다는 거랑 첫문장잡아서 연결지어가면 읽기가 좋았습니다
start
dealing with page 79 3
title: 절대평가시대에 적합한 실전모의고사
이름 : 한재헌   (heon****) 기타
등록일 : 2019-08-19  |  조회 : 237
절대평가로 전환된 이 시점에서 상대평가때와 큰 틀은 바뀌지 않았을지는 몰라도
세부적인것이 많이 변경되었을겁니다.
이 점에서 기출의 누적된 양이 적을 수 밖에 없기에, 조정식 모의고사는
평가원코드에 적합한 실전봉투모의고사입니다.
start
dealing with page 79 4
title: 감자합니다
이름 : 마채웅   (mcw5****) 인문계
등록일 : 2019-08-18  |  조회 : 444
진짜 덕분에 항상 해석해도 안나오던 답이 나오네요
감자감자
start
dealing with page 79 5
title: 모

start
dealing with page 81 3
title: 고2 수강평
이름 : 박수현   (pand****) 자연계
등록일 : 2019-08-15  |  조회 : 199
14강 16분 40초듣고 수강평쓰러 왔습니다. 쌤 인강 14강까지 듣는 친구 개쩐다는 말씀듣고 기분조아서요ㅋㅋ 다니던 과외 끊고 학원으로 옮기려고 하는 그 공백에 영어 감 잃지 않으려고 패스수강으로 선생님 강의 신청하게 되었습니다. 의지 붙들고 괜찮아 완강후에선생님 믿어봐 글읽는법 알려줄게 까지 다 들을려고요. 점점 글이 읽혀지는 느낌이 좋아요 9월 모고보고 좋은 수강평 남겨드리고 싶습니다. 좋은 강의 감사합니다!
start
dealing with page 81 4
title: 조정식 짱
이름 : 이태헌   (thle****) 자연계
등록일 : 2019-08-14  |  조회 : 56
문제 퀄리티 짱 하프모의고사 짱 조정식 짱짱
start
dealing with page 81 5
title: 영어는 조정식!
이름 : 김문희   (ansg****) 자연계
등록일 : 2019-08-14  |  조회 : 39
평가원 모의고사 대비에 조정식모의고사보다 적합한 모의고사는 없는것같다
start
dealing with page 81 6
title: 참 고마운 강좌 ...!...and 정식T두 고마워요
이름 : 국민주   (mjgu****) 인문계
등록일 : 2019-08-14  |  조회 : 72
올해 처음 정식쌤 커리타면서 정식쌤 EBS전지문 듣고
연계체감 어느정도하나 궁금해서 6평대비 모의고사도 사봤는데
음 ... 복습을 너무 안했던 저 자신을 돌아볼수 있는 시험지였고..
강의는 항상 조정식 그자체이며... 난장판이던 6평대비 모의고사 점수덕에..
EBS 열심히 복습하게 되었으며 6평때 처음으로 영어 1을 맞게해준 고마운 강좌입니다
정식쌤은 나의 러닝메이트... 9평대비도 함께 할게요 ㅎㅎㅎㅎ
수능까지 같이 달려요 쌤 ♡ ..

아 ...... 지금 다시 등급이 하락했어용......
9평때까지 

title: 정식이형님인사박습니다
이름 : 전지성   (wjsw****) 
등록일 : 2019-08-12  |  조회 : 94
잘듣고있어영~~ㅎㅎ
속도감있어서좋아요~ㅎㅎ
start
dealing with page 83 10
title: 수능 영어는 단연 조정식 선생님
이름 : 손석민   (msbl****) 인문계
등록일 : 2019-08-12  |  조회 : 39
3월 전 데이 이즈 유얼 데이 특강 이벤트 당첨되어서 조선생님을 뵈러 러셀에 갔었는데 처음 봤을때 그 아우라는 잊을수가 없습니다. 그때 반해서 지금까지 조선생님 믿어봐랑 수능특강 변형문제를 인강으로 듣고 수능완성 변형문제 강의를 현장강의까지 듣고 있습니다. 영어 공부에 시간투자를 많이 하지는 않지만 조쌤 강의 덕분에 2등급을 계속 유지하고 이제는 1등급을 목표로 다시 영어공부를 열심히 하고있습니다. 조쌤 너무 멋있다는 말을 마지막으로 다시 영어공부 열심히 하러 가겠습니다.(꾸벅)
start
dealing with page 84 1
title: 영어는 누구? 조정식샘!!
이름 : 김은규   (dara****) 자연계
등록일 : 2019-08-12  |  조회 : 190
선생님 좋은 강좌 늘 감사합니다 덕분에 모의고사에서 좋은 성과를 거두고 있습니다!!!
start
dealing with page 84 2
title: 어휘의 방향을 잡아줄 강의!
이름 : 김수언   (hi60****) 자연계
등록일 : 2019-08-12  |  조회 : 182
단 한강으로 앞으로의 어휘 학습에 대한 방향과 현재 진행중인 학습법의 문제점을 모두 집어낼 수 있는 좋은 강의입니다.
start
dealing with page 84 3
title: 좋은 기출강좌!
이름 : 김수언   (hi60****) 자연계
등록일 : 2019-08-12  |  조회 : 153
기출분석은 물론이고 단어를 해석하는 눈을 바꾸는 강의입니다.
start
dealing with page 84 4
title: 영어를 처음 공부하는 사람이 보면 좋은 강좌

# 윤훈관 TCC 조회수 가져와~

In [6]:
b.get("https://go3.etoos.com/teacher/tcc/teachertcclist.asp?BOARD_ID=0&PAGE=1&AREA_CD=&TEACHER_ID=200386&ORDER_SEQ=3&SEARCH_TYPE=&SEARCH_WORD=&CONTENTS_GB_CD=&GRADE_CD=0008")

In [20]:
tits = b.find_elements_by_css_selector("ul.tchtcc_list2 > li")

In [24]:
for tit in tits:
    print(tit.text.split('\n'))
    

['▶[특별영상]', '♥ 영어 읽어주기 ASMR ♥ new', '2020.03.06   조회 7118   ']
['▶[학습전략]', '지금 이 시기, 영어 상반기 커리는 이렇게!', '2020.02.28   조회 3613   ']
['▶[특별영상]', "EBS, 윤훈관이 제대로 '찢어'줄게", '2020.02.21   조회 5693   ']
['▶[학습전략]', '수능 31번 지문, 윤훈관 해석법 적용하기', '2020.02.17   조회 5710   ']
['▶[학습전략]', '[노베 공부법] 리스닝편 - 아~! 이게 이런 말이었어?', '2020.02.07   조회 5896   ']
['▶[특별영상]', '암기, 포기하면 편하다! ▶▶ A VOCA DO', '2020.01.31   조회 6614   ']
['▶[학습전략]', '[노베 공부법] 어법편 - 하나를 처음부터 끝까지 조져라!', '2020.01.29   조회 2925   ']
['▶[학습전략]', '[노베 공부법] 어휘편 - 어? 찍었는데 맞았네!', '2020.01.23   조회 6660   ']
['▶[학습전략]', '♣미키마우스편♣ 새로운 해석 짤영상 - 8탄 수동태 해석의 기본적인 관점', '2020.01.17   조회 4599   ']


In [55]:
pages = b.find_elements_by_css_selector('a.link_page')

In [61]:
def tcc_view(browser, worksheet):
    pages = b.find_elements_by_css_selector('a.link_page')
    last_page = int(pages[-1].text)
    page = 1
    while page < last_page+1:
        
        browser.get("https://go3.etoos.com/teacher/tcc/teachertcclist.asp?BOARD_ID=0&PAGE={}&AREA_CD=&TEACHER_ID=200386&ORDER_SEQ=3&SEARCH_TYPE=&SEARCH_WORD=&CONTENTS_GB_CD=&GRADE_CD=0008".format(str(page)))
        info_list = browser.find_elements_by_css_selector("ul.tchtcc_list2 > li")
        for infos in info_list:
            category, title, date_count = infos.text.split('\n')
            date_count = date_count.split(' ')
            date_count = [i for i in date_count if i != '']
            date, count = date_count[0], date_count[2]

            print(category)
            print(title)
            print(date)
            print(count)
            worksheet.append_row([category, title, date, count])
            #print(category, title, date, count)

        page += 1

In [62]:
tcc_view(b, worksheet)

▶[특별영상]
♥ 영어 읽어주기 ASMR ♥ new
2020.03.06
7123
▶[학습전략]
지금 이 시기, 영어 상반기 커리는 이렇게!
2020.02.28
3614
▶[특별영상]
EBS, 윤훈관이 제대로 '찢어'줄게
2020.02.21
5694
▶[학습전략]
수능 31번 지문, 윤훈관 해석법 적용하기
2020.02.17
5710
▶[학습전략]
[노베 공부법] 리스닝편 - 아~! 이게 이런 말이었어?
2020.02.07
5896
▶[특별영상]
암기, 포기하면 편하다! ▶▶ A VOCA DO
2020.01.31
6614
▶[학습전략]
[노베 공부법] 어법편 - 하나를 처음부터 끝까지 조져라!
2020.01.29
2925
▶[학습전략]
[노베 공부법] 어휘편 - 어? 찍었는데 맞았네!
2020.01.23
6660
▶[학습전략]
♣미키마우스편♣ 새로운 해석 짤영상 - 8탄 수동태 해석의 기본적인 관점
2020.01.17
4599
▶[학습전략]
♨핫피플♨ 새로운 해석 짤영상 - 7탄 전치사 해석은 '바로'
2020.01.10
3830
▶[학습전략]
◈조커편◈ 새로운 해석 짤영상 - 6탄 영화 조커 명대사 해석하기
2020.01.06
5114
▶[특별영상]
윤훈관 해석법이 폭발적인 이유
2019.12.31
4686
▶[학습전략]
선생님 해석법이 맞는 것 같은데,, 기존 해석과 충돌이 일어나면 어쩌죠?
2019.12.30
4321
▶[학습전략]
★크리스마스 에디션★ 새로운 해석 짤영상 - 5탄 뭐라구요? 한국식 해석!
2019.12.24
6039
▶[학습전략]
★크리스마스 에디션★ 새로운 해석 짤영상 - 4탄 너에게(X) 너를 향한(O)
2019.12.20
3045
▶[학습전략]
[해석법] 새로운 해석 짤영상 - ３탄 관계사 W의 진실은?
2019.12.13
2327
▶[학습전략]
[윤훈관영어] 2021 연간 커리큘럼 가이드
2019.12.06
5077
▶[학습전략]
[해석법] 새로운 해석 짤영상 - 2탄 5형식 문장 구조는 결국 (S+V)의 반복!
2019.12.05